In [ ]:
# For parse syzbot and crawl the sanitizer input


In [2]:
from glob import glob
import json
import os
from shutil import copyfile


In [3]:
SYZBOT_DATASET_PATH = 'kernel-commitPack/syzbot-crash-commitPack'
SYZBOT_OUTPUT_PATH = "syzbot-data"


In [4]:
def get_popular_keywords():
    keywords = []
    for patch in glob(os.path.join(SYZBOT_DATASET_PATH, '*.json')):
        with open(patch) as f:
            data = json.load(f)
            if data:
                keywords.append(data[0]["title"].split(":")[0])

    # histogram of keywords
    hist = {}
    for keyword in keywords:
        hist[keyword] = hist.get(keyword, 0) + 1

    # sort hist
    return sorted(hist.items(), key=lambda x: x[1], reverse=True)
    
print (get_popular_keywords())


[('KASAN', 776), ('KMSAN', 269), ('BUG', 247), ('WARNING', 206), ('INFO', 197), ('UBSAN', 89), ('kernel panic', 11), ('unregister_netdevice', 7), ('kernel BUG at lib/string.c', 5), ('kernel BUG at drivers/android/binder_alloc.c', 4), ('kernel BUG at arch/x86/mm/physaddr.c', 4), ('kernel BUG at net/core/dev.c', 4), ('suspicious RCU usage at ./include/linux/rcupdate.h', 3), ('kernel BUG at mm/slab.c', 3), ('kernel BUG at drivers/vhost/vhost.c', 2), ('kernel BUG at kernel/time/timer.c', 2), ('kernel BUG at ./include/linux/skbuff.h', 2), ('kernel BUG at fs/userfaultfd.c', 2), ('PANIC', 2), ('kernel BUG at arch/x86/kvm/mmu/mmu.c', 2), ('kernel BUG at net/core/skbuff.c', 2), ('kernel BUG at include/linux/mm.h', 2), ('kernel BUG in __page_mapcount', 1), ('possible deadlock in io_poll_double_wake (3)', 1), ('inconsistent lock state in est_fetch_counters', 1), ('general protection fault in call_commit_handler', 1), ('memory leak in create_io_worker', 1), ('memory leak in ath9k_hif_usb_firmware_

In [10]:
def get_sanitizer_bugtypes(sanitizers=["KASAN", "KMSAN", "UBSAN"]):
    bugtypes = []
    unknown = []
    for patch in glob(os.path.join(SYZBOT_DATASET_PATH, '*.json')):
        with open(patch) as f:
            data = json.load(f)
            if data:
                title = data[0]["title"]
                # title startswith one of the sanitizers
                if any(title.startswith(sanitizer) for sanitizer in sanitizers):
                    bugtype = title.split(" in ")[0].strip().replace(" ", "_").replace(":", "_")
                    bugtypes.append((bugtype, patch))
                else:
                    unknown.append(patch)
            else:
                print (patch, "is empty")
    
    return bugtypes, unknown

bugtypes, unknown = get_sanitizer_bugtypes()


kernel-commitPack/syzbot-crash-commitPack/25554ab865a12b51c66f.json is empty


In [12]:
for bugtype, patch in bugtypes:
    if not os.path.exists(os.path.join(SYZBOT_OUTPUT_PATH, bugtype)):
        os.makedirs(os.path.join(SYZBOT_OUTPUT_PATH, bugtype))

    # copy patch to the output directory
    copyfile(patch, os.path.join(SYZBOT_OUTPUT_PATH, bugtype, os.path.basename(patch)))



In [13]:
for patch in unknown:
    if not os.path.exists(os.path.join(SYZBOT_OUTPUT_PATH, "unknown")):
        os.makedirs(os.path.join(SYZBOT_OUTPUT_PATH, "unknown"))

    # copy patch to the output directory
    copyfile(patch, os.path.join(SYZBOT_OUTPUT_PATH, "unknown", os.path.basename(patch)))
